
# The Mexican Food Resturant Battle of Neighborhoods 
## Part1: Problem Statement and Background:

New york  is the capital city of california. known for many things such as its agriculture, sustainability, infrastructure, healthcare and educational institutions, Sacramento is a lush and vibrant metropolitan city in the northern california.
the neighbborhoods of Sacramento offer a distinct balance between an urbanites dream and suburban escape.

The main purposes for this project is to recommend the best location in Sacramento for new business (Mexican Resturants).


## Part 2:Data:

For the purpose of this project, the data will be restricted to New York City. The information about car dealership will be from the neighborhoods and boroughs of San New York only. We will need a couple of dataset and will combine them to get the desired outcome which are as following datasets:
1)	Sacramento Data (Boroughs +Neighborhoods): https://en.wikipedia.org/wiki/Sacramento,_California  

The first database will be used would contains all requaired geographical data about Sacramento city. namely  we would be used 'Borogh', Latitude, Longitudes, and Neighborhoods among all other data elements presented in our data.
In order to explor/segment the neighborhoods, we will essentially need a dataset that contains the boroghs and th neighborhoods that exist in each borogh as well as the latitude and longitude coordinates for each neighborhood. the "Latitude and "Longitude extracted from this data will also be piovtal when we use it to perform clustering with K-means.

2)	Four Square Sacramento City Guide Data (venues)

Foursquare API: to find dealerships/venues

## Dowonload Important Libraries

In [61]:
!pip install geopy 
!pip install folium
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Libraries imported.


In [62]:
CLIENT_ID = 'ACNVGS3N0ZSQPH5W5RD2ZQEME5LKW1WCOWUDNQUMNTDH12TE' # your Foursquare ID
CLIENT_SECRET = 'ZFX32NLRUT4IWFOU5ZEV0E0NGQK4JZBR2PIPDXPPATTSPBRN' # your Foursquare Secret
ACCESS_TOKEN = 'IEL2M25MWNTDN2QVGKYYL2RBCLZH2GYKHMUFHC4KEFEFU2L5' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ACNVGS3N0ZSQPH5W5RD2ZQEME5LKW1WCOWUDNQUMNTDH12TE
CLIENT_SECRET:ZFX32NLRUT4IWFOU5ZEV0E0NGQK4JZBR2PIPDXPPATTSPBRN


In [64]:
address = 'Sacramento'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.5810606 -121.493895


In [106]:
#sacramento map
sac_map=folium.Map([38.5810606, -121.493895], zoom_start=15)
sac_map

Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~1.5km from Prefecture. Our neighborhoods will be defined as circular areas with a radius of 100 meters, so our neighborhood centers will be 200 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [67]:
search_query = 'Mexican'
radius = 500
print(search_query + ' .... OK!')

Mexican .... OK!


In [68]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ACNVGS3N0ZSQPH5W5RD2ZQEME5LKW1WCOWUDNQUMNTDH12TE&client_secret=ZFX32NLRUT4IWFOU5ZEV0E0NGQK4JZBR2PIPDXPPATTSPBRN&ll=38.5810606,-121.493895&oauth_token=IEL2M25MWNTDN2QVGKYYL2RBCLZH2GYKHMUFHC4KEFEFU2L5&v=20180604&query=Mexican&radius=500&limit=30'

In [69]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60e25a4403cc1d6365c3017b'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '53bed790498e868312f2f169',
    'name': 'Ancho Mexican Grill',
    'location': {'address': '980 9th St Ste 105',
     'crossStreet': 'btwn I & J St',
     'lat': 38.580879517521396,
     'lng': -121.49525758814647,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.580879517521396,
       'lng': -121.49525758814647}],
     'distance': 120,
     'postalCode': '95814',
     'cc': 'US',
     'city': 'Sacramento',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['980 9th St Ste 105 (btwn I & J St)',
      'Sacramento, CA 95814',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1c1941735',
      'name': 'Mexican Restaurant',
      'pluralName': 'Mexican Restaurants',
      'shortName': 'Mexican',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/

In [70]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,53bed790498e868312f2f169,Ancho Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",v-1625446980,False,980 9th St Ste 105,btwn I & J St,38.580880,-121.495258,"[{'label': 'display', 'lat': 38.580879517521396, 'lng': -121.49525758814647}]",120,95814,US,Sacramento,CA,United States,"[980 9th St Ste 105 (btwn I & J St), Sacramento, CA 95814, United States]",NaN,NaN,NaN,NaN,NaN,NaN
1,5632652c498eae6d048d6f0c,Carolina's Mexican Food,"[{'id': '4bf58dd8d48988d153941735', 'name': 'Burrito Place', 'pluralName': 'Burrito Places', 'shortName': 'Burritos', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burrito_', 'suffix': '.png'}, 'primary': True}]",v-1625446980,False,906 J St,NaN,38.580631,-121.494573,"[{'label': 'display', 'lat': 38.58063122255649, 'lng': -121.49457262983208}]",75,95814,US,Sacramento,CA,United States,"[906 J St, Sacramento, CA 95814, United States]",NaN,NaN,NaN,NaN,NaN,NaN
2,55610a46498ee79eb5025fe6,Cilantro's Mexican Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",v-1625446980,False,705 J St,NaN,38.581542,-121.497300,"[{'label': 'display', 'lat': 38.581542000086635, 'lng': -121.49729993818578}]",301,95814,US,Sacramento,CA,United States,"[705 J St, Sacramento, CA 95814, United States]",NaN,NaN,NaN,NaN,NaN,NaN
3,5dfa8d592505200008247dbf,Chipotle Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",v-1625446980,False,615 David J Stern Walk Ste 130,NaN,38.580606,-121.498625,"[{'label': 'display', 'lat': 38.5806059, 'lng': -121.4986251}]",414,95814,US,Sacramento,CA,United States,"[615 David J Stern Walk Ste 130, Sacramento, CA 95814, United States]",NaN,NaN,NaN,NaN,NaN,NaN
4,5b7f05391f7440002cf98037,El Fuego Express Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",v-1625446980,False,1225 J St,NaN,38.579329,-121.489804,"[{'label': 'display', 'lat': 38.579329, 'lng': -121.489804}]",404,95814,US,Sacramento,CA,United States,"[1225 J St, Sacramento, CA 95814, United States]",1071446,https://www.grubhub.com/restaurant/el-fuego-express-mexican-grill-1225-j-st-sacramento/1071446?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=1071446,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png
5,4c635c4f58810f47beb6091e,Tequila Museo Mayahuel,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",v-1625446980,False,1200 K St Ste 3,at 12th St,38.578453,-121.491199,"[{'label': 'display', 'lat': 38.578452838353265, 'lng': -121.49119943858119}]",373,95814,US,Sacramento,CA,United States,"[1200 K St Ste 3 (at 12th St), Sacramento, CA 95814, United States]",2249874,htt

In [71]:
dataframe.shape

(6, 23)

In [74]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Ancho Mexican Grill,Mexican Restaurant,980 9th St Ste 105,btwn I & J St,38.580880,-121.495258,"[{'label': 'display', 'lat': 38.580879517521396, 'lng': -121.49525758814647}]",120,95814,US,Sacramento,CA,United States,"[980 9th St Ste 105 (btwn I & J St), Sacramento, CA 95814, United States]",53bed790498e868312f2f169
1,Carolina's Mexican Food,Burrito Place,906 J St,NaN,38.580631,-121.494573,"[{'label': 'display', 'lat': 38.58063122255649, 'lng': -121.49457262983208}]",75,95814,US,Sacramento,CA,United States,"[906 J St, Sacramento, CA 95814, United States]",5632652c498eae6d048d6f0c
2,Cilantro's Mexican Restaurant,Mexican Restaurant,705 J St,NaN,38.581542,-121.497300,"[{'label': 'display', 'lat': 38.581542000086635, 'lng': -121.49729993818578}]",301,95814,US,Sacramento,CA,United States,"[705 J St, Sacramento, CA 95814, United States]",55610a46498ee79eb5025fe6
3,Chipotle Mexican Grill,Mexican Restaurant,615 David J Stern Walk Ste 130,NaN,38.580606,-121.498625,"[{'label': 'display', 'lat': 38.5806059, 'lng': -121.4986251}]",414,95814,US,Sacramento,CA,United States,"[615 David J Stern Walk Ste 130, Sacramento, CA 95814, United States]",5dfa8d592505200008247dbf
4,El Fuego Express Mexican Grill,Mexican Restaurant,1225 J St,NaN,38.579329,-121.489804,"[{'label': 'display', 'lat': 38.579329, 'lng': -121.489804}]",404,95814,US,Sacramento,CA,United States,"[1225 J St, Sacramento, CA 95814, United States]",5b7f05391f7440002cf98037
5,Tequila Museo Mayahuel,Mexican Restaurant,1200 K St Ste 3,at 12th St,38.578453,-121.491199,"[{'label': 'display', 'lat': 38.578452838353265, 'lng': -121.49119943858119}]",373,95814,US,Sacramento,CA,United States,"[1200 K St Ste 3 (at 12th St), Sacramento, CA 95814, United States]",4c635c4f58810f47beb6091e


In [75]:
dataframe_filtered.name

0    Ancho Mexican Grill           
1    Carolina's Mexican Food       
2    Cilantro's Mexican Restaurant 
3    Chipotle Mexican Grill        
4    El Fuego Express Mexican Grill
5    Tequila Museo Mayahuel        
Name: name, dtype: object

In [76]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) 

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='label',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [77]:
venue_id = '53bed790498e868312f2f169' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/53bed790498e868312f2f169?client_id=ACNVGS3N0ZSQPH5W5RD2ZQEME5LKW1WCOWUDNQUMNTDH12TE&client_secret=ZFX32NLRUT4IWFOU5ZEV0E0NGQK4JZBR2PIPDXPPATTSPBRN&oauth_token=IEL2M25MWNTDN2QVGKYYL2RBCLZH2GYKHMUFHC4KEFEFU2L5&v=20180604'

In [107]:
#result = requests.get(url).json()
#print(result['response']['venue'].keys())
#result['response']['venue']

In [79]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [80]:
venue_id = '5632652c498eae6d048d6f0c' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [81]:
venue_id = '55610a46498ee79eb5025fe6' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [82]:
venue_id = '5dfa8d592505200008247dbf' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.2


In [83]:
venue_id = '5b7f05391f7440002cf98037' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [84]:
venue_id = '4c635c4f58810f47beb6091e' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.4


In [85]:
result['response']['venue']['tips']['count']

42

In [86]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60e25ab96fe9d23a7cab9e61'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 42,
   'items': [{'id': '4fefe847e4b0e65c8d7a1743',
     'createdAt': 1341122631,
     'text': 'Ask for the "Especial de Mexico". It\'s a spicy watermelon margarita-ish cocktail. I know it sounds pretty femme, but this drink is astounding. Never had anything like it. Believe.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/4fefe847e4b0e65c8d7a1743',
     'likes': {'count': 2,
      'groups': [{'type': 'others',
        'count': 2,
        'items': [{'id': '1524861',
          'firstName': 'Nick',
          'lastName': 'Teykaerts',
          'gender': 'male',
          'countryCode': 'US',
          'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
           'suffix': '/1524861-FZXNSRXZLJO00HQG.jpg'}},
         {'id': '24806343',
          'firstName': 'Nadia',
          'lastName':

In [88]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'likes', 'like', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

In [89]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if __name__ == '__main__':
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"Ask for the ""Especial de Mexico"". It's a spicy watermelon margarita-ish cocktail. I know it sounds pretty femme, but this drink is astounding. Never had anything like it. Believe.",3,0,4fefe847e4b0e65c8d7a1743,ᴡ,Teykaerts,1543800


In [90]:
idnumber = '1543800' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}/?client_id={}&client_secret={}&oauth_token={}&v={}'.format(idnumber,CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN,VERSION) # define URL

# send GET request
results = requests.get(url).json()

user_data=results['response']['user']['photos']['items']

#results
pd.set_option('display.max_colwidth', -1)

users_df = json_normalize(user_data)
#This mainly used later to display the photo of the user
filtered_columns = ['id','prefix','suffix','width','height']
tips_filtered = users_df.loc[:, filtered_columns]
#url
tips_filtered

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,prefix,suffix,width,height
0,5df3b9ee775cb400086e4fac,https://fastly.4sqi.net/img/general/,/1543800_566t-eolMR9xzfJJi_s0aq_TveoImeJT35X9JJW_Mg8.jpg,1440,1920


In [91]:
g=tips_df.loc[tips_df['user.id'] == '1543800']
print('First Name: ' + tips_df['user.firstName'])
print('Last Name: ' + tips_df['user.lastName'])

0    First Name: ᴡ
Name: user.firstName, dtype: object
0    Last Name: Teykaerts
Name: user.lastName, dtype: object


In [92]:
# define tips URL
user_id='1543800'
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN,VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,text,agreeCount,disagreeCount,id
0,"YOU SHALL NOT PASS up Balrog Coffee, it’s great",0,0,5e32e4a6e06b6b00069dc54e
1,Best NYC style pizza in town. Will not disappoint.,0,0,5e319ceac2518f0006428008
2,World class selection of whisk(e)y from all over the world. Appreciate 1.5/2.5oz pour options. Prices are reasonable for the selection and quality pours.,0,0,5e2afed8b65f6900076261b7
3,The handle of the coffee mug is too narrow for the average human finger,0,0,5de540af0e5a3200073e6132
4,Tip: don’t eat the pizza.,0,1,5cae63bbe96d0c002c99238c
5,"$24 for 20 delicious wings, after tax. Good deal, great chicken.",0,0,5c6e13ab872f7d002cae8886
6,Get the Old Fashioned,0,0,5c64e180b6b04b002cb8f836
7,Get the Bee 🐝,1,0,5ba3c1d12b9844002c1b5f8a
8,Check out the sour gummies behind the counter,1,0,5b8c27ac8194fc002ca07223
9,Only place in town you can routinely find not just one but THREE Belgian quads on tap,0,0,5b69b46da795bd002c6b5441


In [93]:
tip_id = '5e32e4a6e06b6b00069dc54e' # tip id

# define URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION) # define URL


# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tips']['items'][0]['venue']['name'])
print(result['response']['tips']['items'][0]['venue']['location'])

Balrog
{'lat': 33.788322, 'lng': -118.10864, 'labeledLatLngs': [{'label': 'display', 'lat': 33.788322, 'lng': -118.10864}], 'postalCode': '90815', 'cc': 'US', 'neighborhood': 'Cal State University Long Beach', 'city': 'Long Beach', 'state': 'CA', 'country': 'United States', 'formattedAddress': ['Long Beach, CA 90815', 'United States']}


In [94]:
latitude=38.5810606 
longitude=-121.493895

In [109]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=ACNVGS3N0ZSQPH5W5RD2ZQEME5LKW1WCOWUDNQUMNTDH12TE&client_secret=ZFX32NLRUT4IWFOU5ZEV0E0NGQK4JZBR2PIPDXPPATTSPBRN&ll=38.5810606,-121.493895&v=20180604&radius=500&limit=30'

In [110]:
import requests

In [111]:
results = requests.get(url).json()
'There are {} around TEQUILA MUSEO  restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around TEQUILA MUSEO  restaurant.'

In [98]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b5dbd50f964a5204d6929e3',
  'name': 'Temple Coffee & Tea',
  'location': {'address': '1010 9th St',
   'crossStreet': 'btwn J & K St',
   'lat': 38.58041379940968,
   'lng': -121.49503550282543,
   'labeledLatLngs': [{'label': 'display',
     'lat': 38.58041379940968,
     'lng': -121.49503550282543}],
   'distance': 122,
   'postalCode': '95814',
   'cc': 'US',
   'neighborhood': 'Downtown Sacramento',
   'city': 'Sacramento',
   'state': 'CA',
   'country': 'United States',
   'formattedAddress': ['1010 9th St (btwn J & K St)',
    'Sacramento, CA 95814',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},

In [99]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Temple Coffee & Tea,Coffee Shop,1010 9th St,btwn J & K St,38.580414,-121.495036,"[{'label': 'display', 'lat': 38.58041379940968, 'lng': -121.49503550282543}]",122,95814,US,Downtown Sacramento,Sacramento,CA,United States,"[1010 9th St (btwn J & K St), Sacramento, CA 95814, United States]",4b5dbd50f964a5204d6929e3
1,Bud's Buffet,Sandwich Place,1016 10th St,at J St,38.579657,-121.493801,"[{'label': 'display', 'lat': 38.57965739761994, 'lng': -121.49380099125635}]",156,95814,US,NaN,Sacramento,CA,United States,"[1016 10th St (at J St), Sacramento, CA 95814, United States]",4b47a3b2f964a520963826e3
2,"The Citizen Hotel, Autograph Collection",Hotel,926 J Street,at 10th St,38.580140,-121.493857,"[{'label': 'display', 'lat': 38.58014001108063, 'lng': -121.49385712576077}]",102,95814,US,NaN,Sacramento,CA,United States,"[926 J Street (at 10th St), Sacramento, CA 95814, United States]",4b0586b3f964a5207a6a22e3
3,Coin-Op Game Room,Bar,908 K St,NaN,38.579388,-121.495107,"[{'label': 'display', 'lat': 38.57938820063236, 'lng': -121.49510668533865}, {'label': 'entrance', 'lat': 38.579213, 'lng': -121.495269}]",213,95814,US,NaN,Sacramento,CA,United States,"[908 K St, Sacramento, CA 95814, United States]",554be0a2498e1edf55515978
4,Miosa Bride,Bridal Shop,1125 J St,at 12th St,38.579924,-121.491045,"[{'label': 'display', 'lat': 38.5799242, 'lng': -121.4910454}]",278,95814,US,NaN,Sacramento,CA,United States,"[1125 J St (at 12th St), Sacramento, CA 95814, United States]",4b9ad151f964a520b7d835e3
5,Grange,New American Restaurant,926 J St,at 10th St,38.580250,-121.493686,"[{'label': 'display', 'lat': 38.580250142958086, 'lng': -121.49368554353714}]",92,95814,US,NaN,Sacramento,CA,United States,"[926 J St (at 10th St), Sacramento, CA 95814, United States]",4add393af964a520a96421e3
6,Crest Theatre,Movie Theater,1013 K St,at 11th St,38.579126,-121.493297,"[{'label': 'display', 'lat': 38.5791261146081, 'lng': -121.49329688152058}, {'label': 'entrance', 'lat': 38.579269, 'lng': -121.493238}]",221,95814,US,NaN,Sacramento,CA,United States,"[1013 K St (at 11th St), Sacramento, CA 95814, United States]",4b0586baf964a5207e6b22e3
7,The Boiling Crab,Seafood Restaurant,1000 K St,NaN,38.578976,-121.493678,"[{'label': 'display', 'lat': 38.57897581218635, 'lng': -121.49367792185204}]",232,95814,US,Downtown Sacramento,Sacramento,CA,United States,"[1000 K St, Sacramento, CA 95814, United States]",585c7f9c2b04f80cddc69731
8,Ambrosia Cafe,Café,1030 K St,at 11th St,38.578843,-121.492846,"[{'label': 'display', 'lat': 38.57884256915307, 'lng': -121.49284635409043}]",263,95814,US,NaN,Sacramento,CA,United States,"[1030 K St (at 11th St), Sacramento, CA 95814, United States]",4b05d229f964a520ebe322e3
9,Crest Cafe,Mediterranean Restaurant,1017 K St,NaN,38.579039,-121.493249,"[{'label': 'display', 'lat': 38.57903859713826, 'lng': -121.49324933622314}]",231,95814,US,NaN,Sacramento,CA,United States,"[1017 K St, Sacramento, CA 95814, United States]",4b98168af964a5207b2b35e3


In [112]:
dataframe_filtered.name

0     Temple Coffee & Tea                    
1     Bud's Buffet                           
2     The Citizen Hotel, Autograph Collection
3     Coin-Op Game Room                      
4     Miosa Bride                            
5     Grange                                 
6     Crest Theatre                          
7     The Boiling Crab                       
8     Ambrosia Cafe                          
9     Crest Cafe                             
10    California Family Fitness              
11    Ella Dining Room & Bar                 
12    Social Nightclub                       
13    Frank Fat's                            
14    Dad's On J                             
15    Pizza Rock                             
16    Esquire IMAX Theatre                   
17    See's Candies                          
18    Bangkok@12                             
19    Hyatt Regency Sacramento               
20    Starbucks Reserve                      
21    Kimpton Sawyer Hotel        

In [100]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

In [39]:
venues_map

In [101]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60e25b1fa8f5af04c370058b'},
 'response': {'venues': []}}

In [102]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [103]:
trending_venues_df

'No trending venues are available at the moment!'

In [104]:
if len(results['response']['venues']) == 0:
    trending_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    trenging_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(trending_map)

In [105]:
trending_map

'Cannot generate visual as no trending venues are available at the moment!'

# the end